# 비정형 데이터 전처리

In [1]:
import os
import numpy as np
import pandas as pd

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from nltk import ngrams

In [2]:
os.getcwd()

'c:\\git\\DataAnalysisProcess\\code'

In [3]:
os.chdir('../data')

In [4]:
os.listdir()

['APT_Price_2023.xlsx',
 'iris.csv',
 'iris.json',
 'iris.parquet',
 'iris.tsv',
 'iris.xlsx',
 'Stock_News_202301.xlsx']

In [5]:
df = pd.read_excel(io = 'Stock_News_202301.xlsx')

df.head()

,일자,제목,본문,링크
0,2023-01-01,침체 우려 속 ‘1월 효과’ 기대... 美 금리 방향성 가장 큰 변수 [주간 증시 전망],\n\t\t\t\t\t\t\t\t\t\t1월 첫주 증시는 연말 증시 부진을 만회할 ...,/news/news_read.naver?article_id=0004949843&of...
1,2023-01-01,"[펀드와치]급락장에 웃은 코스닥 인버스…""1월 효과 주목""",\n\t\t\t\t\t\t\t\t\t\t막주 국내 증시 인플레 부담·긴축·배당락에 ...,/news/news_read.naver?article_id=0005397624&of...
2,2023-01-01,[주간증시전망]1월효과에 쏠리는 눈…종목장세 전망,\n\t\t\t\t\t\t\t\t\t\t경기침체 우려에 1월효과 '중립' 무게장단기...,/news/news_read.naver?article_id=0005397588&of...
3,2023-01-01,"'최악 2022' 넘긴 美증시, 새해 상반기 더 나쁠 수 있지만…[월가시각]",\n\t\t\t\t\t\t\t\t\t\t['상저하고' 전망이 대세… 관건은 경기침체...,/news/news_read.naver?article_id=0004835033&of...
4,2023-01-02,[마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환,\n\t\t\t\t\t\t\t\t\t\t시총 상위 종목 10개는 모두 상승 현대·...,/news/news_read.naver?article_id=0000866844&of...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      257 non-null    object
 1   제목      257 non-null    object
 2   본문      257 non-null    object
 3   링크      257 non-null    object
dtypes: object(4)
memory usage: 8.2+ KB


In [7]:
sens = df['제목']

In [8]:
sens.head()

0    침체 우려 속 ‘1월 효과’ 기대... 美 금리 방향성 가장 큰 변수 [주간 증시 전망]
1                     [펀드와치]급락장에 웃은 코스닥 인버스…"1월 효과 주목"
2                          [주간증시전망]1월효과에 쏠리는 눈…종목장세 전망
3           '최악 2022' 넘긴 美증시, 새해 상반기 더 나쁠 수 있지만…[월가시각]
4        [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환
Name: 제목, dtype: object

### 텍스트 전처리
1. 알파벳 소문자를 대문자로 변경
2. 홈페이지, 날짜, 전화번호, 이메일 패턴 등을 추출하거나 삭제
3. 한글 자음과 모음을 삭제
4. 불필요한 기호 및 띄어쓰기 삭제 (오타자 형태소 분석 과정에서 처리)

In [9]:
strs = pd.Series(data = ['abcd', 'ABCD', '1234', '가나다라', 'ㅋㅎㅜㅠ',',:%#'])

In [10]:
strs

0    abcd
1    ABCD
2    1234
3    가나다라
4    ㅋㅎㅜㅠ
5    ,:%#
dtype: object

In [11]:
# 불필요한 한글 자음 모음과 특수문자 제외
strs.str.extractall(pat = '([^a-zA-Z0-9가-힣])')

0
  match   
4 0      ㅋ
  1      ㅎ
  2      ㅜ
  3      ㅠ
5 0      ,
  1      :
  2      %
  3      #

In [12]:
sens.head()

0    침체 우려 속 ‘1월 효과’ 기대... 美 금리 방향성 가장 큰 변수 [주간 증시 전망]
1                     [펀드와치]급락장에 웃은 코스닥 인버스…"1월 효과 주목"
2                          [주간증시전망]1월효과에 쏠리는 눈…종목장세 전망
3           '최악 2022' 넘긴 美증시, 새해 상반기 더 나쁠 수 있지만…[월가시각]
4        [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환
Name: 제목, dtype: object

In [13]:
# 한자만 검색(한 일 - 부를 유)
sens.str.extractall(pat = '([一-龥])')

0
    match   
0   0      美
3   0      美
9   0      韓
    1      日
    2      中
...       ..
247 0      中
248 0      美
251 0      三
    1      電
255 0      美

[132 rows x 1 columns]

In [14]:
# transfer Series & value count
sens.str.extractall(pat = '([一-龥])')[0].value_counts()

0
美    58
中    23
日    18
英     8
株     7
獨     5
外     2
人     2
三     2
電     2
韓     1
發     1
弗     1
尹     1
訴     1
Name: count, dtype: int64

In [15]:
sens.str.replace(pat = '美', repl = '미국')

0      침체 우려 속 ‘1월 효과’ 기대... 미국 금리 방향성 가장 큰 변수 [주간 증시...
1                       [펀드와치]급락장에 웃은 코스닥 인버스…"1월 효과 주목"
2                            [주간증시전망]1월효과에 쏠리는 눈…종목장세 전망
3            '최악 2022' 넘긴 미국증시, 새해 상반기 더 나쁠 수 있지만…[월가시각]
4          [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환
                             ...                        
252                         [코스닥 마감]외국인 매수에 소폭 상승…0.25%↑
253                     [Asia오전] 연준 발표를 기다려볼까? 방향감 잃은 증시
254             FOMC 목전 '위험선호 위축'…환율 1230원대 상향 시도[외환브리핑]
255          FOMC 앞둔 미국증시 하락…삼성전자 '반도체 감산' 여부 주목 [모닝브리핑]
256             달리던 나스닥, 연준 눈치에 급락…'전기차 인하' 포드 뚝↓ [뉴욕마감]
Name: 제목, Length: 257, dtype: object

In [16]:
sens.head()

0    침체 우려 속 ‘1월 효과’ 기대... 美 금리 방향성 가장 큰 변수 [주간 증시 전망]
1                     [펀드와치]급락장에 웃은 코스닥 인버스…"1월 효과 주목"
2                          [주간증시전망]1월효과에 쏠리는 눈…종목장세 전망
3           '최악 2022' 넘긴 美증시, 새해 상반기 더 나쁠 수 있지만…[월가시각]
4        [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환
Name: 제목, dtype: object

In [17]:
sens = sens.str.replace(pat = '([^a-zA-Z0-9가-힣])', repl='', regex=True)
sens

0          침체우려속1월효과기대금리방향성가장큰변수주간증시전망
1               펀드와치급락장에웃은코스닥인버스1월효과주목
2                주간증시전망1월효과에쏠리는눈종목장세전망
3           최악2022넘긴증시새해상반기더나쁠수있지만월가시각
4      마켓뷰새해첫거래코스피코스닥모두하락외국인기관순매도에약세전환
                    ...               
252                 코스닥마감외국인매수에소폭상승025
253            Asia오전연준발표를기다려볼까방향감잃은증시
254      FOMC목전위험선호위축환율1230원대상향시도외환브리핑
255       FOMC앞둔증시하락삼성전자반도체감산여부주목모닝브리핑
256          달리던나스닥연준눈치에급락전기차인하포드뚝뉴욕마감
Name: 제목, Length: 257, dtype: object

### 형태소 분석

In [18]:
sen = '복리후생으로워라밸부터챙기잨ㅋㅋㅋ'

In [19]:
# 한글 형태소 분석기 설정
kiwi = Kiwi()

# 형태소 분석
kiwi.tokenize(text = sen)

[Token(form='복리', tag='NNG', start=0, len=2),
 Token(form='후생', tag='NNG', start=2, len=2),
 Token(form='으로', tag='JKB', start=4, len=2),
 Token(form='워라밸', tag='NNG', start=6, len=3),
 Token(form='부터', tag='JX', start=9, len=2),
 Token(form='챙기', tag='VV', start=11, len=2),
 Token(form='자', tag='EC', start=13, len=1),
 Token(form='ᆿ', tag='Z_CODA', start=13, len=1),
 Token(form='ㅋㅋㅋ', tag='SW', start=14, len=3)]

### 불용어 삭제

In [20]:
# 내장 불용어 객체를 생성
stopwords = Stopwords()

# 내장 불용어 목록을 확인
stopwords.stopwords


{('ᆫ', 'ETM'),
 ('ᆫ', 'JX'),
 ('ᆫ다', 'EF'),
 ('ᆯ', 'ETM'),
 ('가', 'JKS'),
 ('같', 'VA'),
 ('것', 'NNB'),
 ('게', 'EC'),
 ('겠', 'EP'),
 ('고', 'EC'),
 ('고', 'JKQ'),
 ('과', 'JC'),
 ('과', 'JKB'),
 ('그', 'MM'),
 ('그', 'NP'),
 ('기', 'ETN'),
 ('까지', 'JX'),
 ('나', 'NP'),
 ('년', 'NNB'),
 ('는', 'ETM'),
 ('는', 'JX'),
 ('다', 'EC'),
 ('다', 'EF'),
 ('다고', 'EC'),
 ('다는', 'ETM'),
 ('대하', 'VV'),
 ('더', 'MAG'),
 ('던', 'ETM'),
 ('도', 'JX'),
 ('되', 'VV'),
 ('되', 'XSV'),
 ('들', 'XSN'),
 ('등', 'NNB'),
 ('따르', 'VV'),
 ('때', 'NNG'),
 ('때문', 'NNB'),
 ('라', 'EC'),
 ('라는', 'ETM'),
 ('로', 'JKB'),
 ('를', 'JKO'),
 ('만', 'JX'),
 ('만', 'NR'),
 ('말', 'NNG'),
 ('며', 'EC'),
 ('면', 'EC'),
 ('면서', 'EC'),
 ('명', 'NNB'),
 ('받', 'VV'),
 ('보', 'VV'),
 ('부터', 'JX'),
 ('사람', 'NNG'),
 ('성', 'XSN'),
 ('수', 'NNB'),
 ('아니', 'VCN'),
 ('않', 'VX'),
 ('어', 'EC'),
 ('어', 'EF'),
 ('어서', 'EC'),
 ('어야', 'EC'),
 ('없', 'VA'),
 ('었', 'EP'),
 ('에', 'JKB'),
 ('에게', 'JKB'),
 ('에서', 'JKB'),
 ('와', 'JC'),
 ('와', 'JKB'),
 ('우리', 'NP'),
 ('원', 'NNB'),


In [21]:
# 형태소 분석 결과에서 불용어를 제거
kiwi.tokenize(text = sen, stopwords = stopwords)

[Token(form='복리', tag='NNG', start=0, len=2),
 Token(form='후생', tag='NNG', start=2, len=2),
 Token(form='워라밸', tag='NNG', start=6, len=3),
 Token(form='챙기', tag='VV', start=11, len=2),
 Token(form='자', tag='EC', start=13, len=1),
 Token(form='ᆿ', tag='Z_CODA', start=13, len=1)]

In [22]:
# 사용자 사전에 특정 단어 추가
kiwi.add_user_word(word='복리후생', tag = 'NNG', score=15)

True

In [23]:
# 형태소 분석 결과에서 불용어를 제거
tokens = kiwi.tokenize(text = sen, stopwords = stopwords)

### 형태소 분석 결과 확인

In [24]:
# tokens(형태소 분석 결과)를 확인
tokens[0]

Token(form='복리후생', tag='NNG', start=0, len=4)

In [25]:
# 단어
tokens[0].form

'복리후생'

In [26]:
# 품사
tokens[0].tag

'NNG'

### 품사 선택

In [27]:
# 텍스트 분석에 사용할 품사(용언과 체언) 목록을 리스트로 생성
pos1, pos2 = ['VV', 'VA'], ['NNG', 'NNP']

In [28]:
# 형태소의 품사가 용언과 체언인 형태소만 선택 후, 품사가 용언인 형태소에 종결어미 '다'를 결합하여 리스트로 반환
tokens = [token.form + '다' if token.tag in pos1 else token.form for token in tokens if token.tag in pos1 + pos2]

tokens

['복리후생', '워라밸', '챙기다']

### 말뭉치 생성

In [29]:
# 말뭉치를 저장할 빈 리스트 생성
corpus = list()

In [30]:
for sen in sens:
    tokens = kiwi.tokenize(text = sen, stopwords = stopwords)
    tokens = [token.form + '다' if token.tag in pos1 else token.form for token in tokens if token.tag in pos1 + pos2]
    corpus.append(tokens)

In [31]:
df['제목'].head()

0    침체 우려 속 ‘1월 효과’ 기대... 美 금리 방향성 가장 큰 변수 [주간 증시 전망]
1                     [펀드와치]급락장에 웃은 코스닥 인버스…"1월 효과 주목"
2                          [주간증시전망]1월효과에 쏠리는 눈…종목장세 전망
3           '최악 2022' 넘긴 美증시, 새해 상반기 더 나쁠 수 있지만…[월가시각]
4        [마켓뷰] 새해 첫 거래 코스피·코스닥 모두 하락…외국인·기관 순매도에 약세 전환
Name: 제목, dtype: object

#### 원본 문서와 형태소 분석 결과 비교

In [32]:
corpus[0:5]

[['침체', '우려', '속', '효과기', '금리', '방향', '크다', '변수', '주간', '증시', '전망'],
 ['펀드', '치', '급락', '장', '코스닥', '버스', '효과', '주목'],
 ['주간', '증시', '전망', '효과', '쏠리다', '눈', '종목', '장세', '전망'],
 ['최악', '넘기다', '증시', '새해', '상반기', '나쁘다', '있다', '월', '시각'],
 ['마켓', '뷰', '새해', '거래', '코스피', '코스닥', '하락', '외국인', '기관', '순매도', '약세', '전환']]

In [33]:
# 사용자 사전에 추가할 단어 목록
words = ['1월효과', '방향성', '펀드와치', '급락장', '인버스', '월가']

# 사용자 사전에 단어 추가
for word in words:
    kiwi.add_user_word(word = word, tag = 'NNG', score=15)

In [34]:
# 형태소 재분석 하기
corpus = list()
for sen in sens:
    tokens = kiwi.tokenize(text = sen, stopwords = stopwords)
    tokens = [token.form + '다' if token.tag in pos1 else token.form for token in tokens if token.tag in pos1 + pos2]
    corpus.append(tokens)

In [35]:
corpus[0:5]

[['침체', '우려', '속', '1월효과', '기대', '금리', '방향성', '크다', '변수', '주간', '증시', '전망'],
 ['펀드와치', '급락장', '코스닥', '인버스', '1월효과', '주목'],
 ['주간', '증시', '전망', '1월효과', '쏠리다', '눈', '종목', '장세', '전망'],
 ['최악', '넘기다', '증시', '새해', '상반기', '나쁘다', '있다', '월가', '시각'],
 ['마켓', '뷰', '새해', '거래', '코스피', '코스닥', '하락', '외국인', '기관', '순매도', '약세', '전환']]

### n-gram

In [36]:
bigram = ngrams(sequence=corpus[0], n =2)

In [37]:
corpus[0]

['침체', '우려', '속', '1월효과', '기대', '금리', '방향성', '크다', '변수', '주간', '증시', '전망']

In [38]:
bigram

In [39]:
list(bigram)

[('침체', '우려'),
 ('우려', '속'),
 ('속', '1월효과'),
 ('1월효과', '기대'),
 ('기대', '금리'),
 ('금리', '방향성'),
 ('방향성', '크다'),
 ('크다', '변수'),
 ('변수', '주간'),
 ('주간', '증시'),
 ('증시', '전망')]

### corpus로 bigrams 생성

In [40]:
bigrams = list()

In [41]:
for i in range(len(corpus)):
    bigram = ngrams(sequence=corpus[i], n=2)
    bigrams += list(bigram)

bigrams

[('침체', '우려'),
 ('우려', '속'),
 ('속', '1월효과'),
 ('1월효과', '기대'),
 ('기대', '금리'),
 ('금리', '방향성'),
 ('방향성', '크다'),
 ('크다', '변수'),
 ('변수', '주간'),
 ('주간', '증시'),
 ('증시', '전망'),
 ('펀드와치', '급락장'),
 ('급락장', '코스닥'),
 ('코스닥', '인버스'),
 ('인버스', '1월효과'),
 ('1월효과', '주목'),
 ('주간', '증시'),
 ('증시', '전망'),
 ('전망', '1월효과'),
 ('1월효과', '쏠리다'),
 ('쏠리다', '눈'),
 ('눈', '종목'),
 ('종목', '장세'),
 ('장세', '전망'),
 ('최악', '넘기다'),
 ('넘기다', '증시'),
 ('증시', '새해'),
 ('새해', '상반기'),
 ('상반기', '나쁘다'),
 ('나쁘다', '있다'),
 ('있다', '월가'),
 ('월가', '시각'),
 ('마켓', '뷰'),
 ('뷰', '새해'),
 ('새해', '거래'),
 ('거래', '코스피'),
 ('코스피', '코스닥'),
 ('코스닥', '하락'),
 ('하락', '외국인'),
 ('외국인', '기관'),
 ('기관', '순매도'),
 ('순매도', '약세'),
 ('약세', '전환'),
 ('시황', '종합'),
 ('종합', '코스피'),
 ('코스피', '개장'),
 ('개장', '내리다'),
 ('마감', '시황'),
 ('시황', '코스피'),
 ('코스피', '새해'),
 ('새해', '첫날'),
 ('첫날', '기관'),
 ('기관', '매물'),
 ('새해', '첫날'),
 ('첫날', '환율'),
 ('환율', '저가'),
 ('저가', '매수'),
 ('매수', '원'),
 ('원', '거래일'),
 ('거래일', '상승'),
 ('상승', '외환'),
 ('외환', '마감'),
 ('코스닥', '마감'),
 ('마감', '하락'),
 (

In [42]:
len(bigrams)

1885

In [43]:
bigrams = pd.Series(data = bigrams)

In [44]:
bigrams.value_counts()

(뉴욕, 마감)     30
(외환, 마감)     20
(외환, 브리핑)    20
(시황, 종합)     20
(마감, 시황)     19
             ..
(물가, 실적)      1
(조, 이재명)      1
(이재명, 검찰)     1
(검찰, 출석)      1
(포드, 뉴욕)      1
Name: count, Length: 1190, dtype: int64

In [45]:
# 한단어로 만들 수 있는 단어 조합을 찾는 Tip
bigrams.head(n=30)

0        (침체, 우려)
1         (우려, 속)
2       (속, 1월효과)
3      (1월효과, 기대)
4        (기대, 금리)
5       (금리, 방향성)
6       (방향성, 크다)
7        (크다, 변수)
8        (변수, 주간)
9        (주간, 증시)
10       (증시, 전망)
11    (펀드와치, 급락장)
12     (급락장, 코스닥)
13     (코스닥, 인버스)
14    (인버스, 1월효과)
15     (1월효과, 주목)
16       (주간, 증시)
17       (증시, 전망)
18     (전망, 1월효과)
19    (1월효과, 쏠리다)
20       (쏠리다, 눈)
21        (눈, 종목)
22       (종목, 장세)
23       (장세, 전망)
24      (최악, 넘기다)
25      (넘기다, 증시)
26       (증시, 새해)
27      (새해, 상반기)
28     (상반기, 나쁘다)
29      (나쁘다, 있다)
dtype: object

In [47]:
# 사용자 사전에 추가할 단어 목록
words = ['마켓뷰', '뉴욕증시']

# 사용자 사전에 단어 추가
for word in words:
    kiwi.add_user_word(word = word, tag = 'NNG', score=15)